This code reads in a csv file containing data on the utilization of buses in various municipalities, preprocesses the data, and performs time series forecasting using the Simple Exponential Smoothing method, SARIMA and Exponential Smoothing for each municipality. Statsmodels and Scikit-learn libraries are used for time series analysis and evaluation.

The first few lines of code import the necessary libraries and suppress warning messages.

The next section reads in the csv file containing the data and prints the first 30 rows to check the data. It also visualizes the data using a barplot to show the usage per municipality.

The "Data preprocessing and feature extraction" section contains functions that extract date features from the timestamp data, resample the data in hourly basis, divide the data into train and test splits for each municipality, and find the best alpha value for each municipality. Alpha is a parameter in the Simple Exponential Smoothing method that controls the level of smoothing applied to the time series. The section also contains a function to plot the forecasted time series data.

Finally, the code loops through each municipality and finds the best alpha value using the alpha_validation function, trains a Simple Exponential Smoothing model on the training data with the best alpha value, generates forecasts for the test data, and plots the forecasted time series data using the plot_prediction function.

Then code repeats loops through SARIMA and Exponential Smoothing as well


In [ ]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import itertools

from statsmodels.tsa.holtwinters  import SimpleExpSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters  import ExponentialSmoothing

from sklearn.metrics import mean_absolute_error

In [ ]:
"""Read data"""
df = pd.read_csv("C:\\Users\\Serkan\\Downloads\\municipality_bus_utilization.csv", parse_dates=['timestamp'])

#Print first 30 elements to check the data
print(df.head(30))

In [ ]:
"""Visualize data to have a better understanding"""
sns.set_style("whitegrid")
plt.figure(figsize = (8, 4))
sns.barplot(x = df["municipality_id"], y = df["usage"])
plt.xlabel('Municipality')
plt.ylabel('Usage')
plt.title('Usage per municipalities')
plt.show()

In [ ]:
"""Data preprocessing and featrue extraction"""

#First get every time information from data such as hours and days and check for missing values
def create_date_features(df):
    df['hour'] = df.timestamp.dt.hour
    df['month'] = df.timestamp.dt.month
    df['day_of_month'] = df.timestamp.dt.day
    df['day_of_year'] = df.timestamp.dt.dayofyear
    df['week_of_year'] = df.timestamp.dt.weekofyear
    df['day_of_week'] = df.timestamp.dt.dayofweek
    df['year'] = df.timestamp.dt.year
    df["is_wknd"] = df.timestamp.dt.weekday // 4
    df['is_month_start'] = df.timestamp.dt.is_month_start.astype(int)
    df['is_month_end'] = df.timestamp.dt.is_month_end.astype(int)
    return df
df = create_date_features(df)
df.isnull().sum() # show missing values

df.groupby(["municipality_id","hour"]).agg({"usage": ["count", "max"]})

# All the municipalities have the same pattern. We don't need to deal with the missing hours.

In [ ]:
"""Resample data in hourly basis"""

df_resampled = pd.DataFrame()

df["timestamp"] = df["timestamp"].astype(str).apply(lambda x: x[:-6]).astype("datetime64")
# Get the max value for the duplicates as recommended in the task
df_resampled = df.groupby(["timestamp","municipality_id"]).agg({"usage": "max"}).reset_index()
df_resampled.drop_duplicates(["timestamp","municipality_id"],inplace=True)

df_resampled.head()

#Store the resampled version for each municipality in order separately
df_ordered={}
for i in range(10):
    df_ordered[i]= pd.DataFrame(data=df_resampled[df_resampled.municipality_id==i], columns=["timestamp","usage"]).set_index("timestamp")
    

In [ ]:
#Divide the data into train and test split for each municipality
train_data = {}
test_data = {}
train_end_date = "2017-08-04 16:00:00"
test_start_date = "2017-08-05 07:00:00"
for i in range(10):
    train_data[i] = df_ordered[i][:train_end_date]
    test_data[i] = df_ordered[i][test_start_date:]
    

SIMPLE EXPONENTIAL SMOOTHING

In [ ]:
"""SIMPLE EXPONENTIAL SMOOTHING"""
"""Create hyperparameter finder function and plot function"""

#We need to pick the best alpha for each municipality before training with a fixed alpha
#For this reason create a function that finds the best alpha value in a range
def alpha_validation(train,test, alphas, step=142):
    best_alpha, lowest_mae = None, float("inf")
    for alpha in alphas:
        valid_model = SimpleExpSmoothing(train).fit(smoothing_level=alpha)
        y_pred = valid_model.forecast(step)
        mae = mean_absolute_error(test, y_pred)
        if mae < lowest_mae: # choose the alpha with lowest mae value
            best_alpha, lowest_mae = alpha, mae
    print("Best alpha is:", round(best_alpha, 2), "and lowest MAE is:", round(lowest_mae, 4))
    return best_alpha, lowest_mae

def plot_prediction(i,y_pred):
    plt.figure(figsize=(16, 4))
    train_data[i]["usage"].plot(legend=True, label=f"TRAIN {i}")
    test_data[i]["usage"].plot(legend=True, label=f"TEST {i}")
    if "usage" in y_pred:
        y_pred["usage"].plot(legend=True, label=f"PREDICTION {i}")
    else:
        y_pred["predicted_mean"].plot(legend=True, label=f"PREDICTION {i}") # This line is for the values coming from SARIMA
    plt.xlim([datetime.date(2017,6,4), datetime.date(2017,8,20)])
    plt.title("Time Series Forecasting for Bus Demands in Municipality " +str(i))
    plt.show()

In [ ]:
alphas = np.arange(0.01, 1, 0.10)
for i in range(10):   
    best_alpha, _ = alpha_validation(train_data[i],test_data[i], alphas, step=142)
    test_model = SimpleExpSmoothing(train_data[i]).fit(smoothing_level=best_alpha)
    y_pred = test_model.forecast(142)

    y_pred.reset_index(drop=True,inplace=True)
    y_pred=pd.DataFrame(y_pred, columns=["usage"])
    y_pred = y_pred.merge(test_data[i].reset_index()["timestamp"], left_index=True, right_index=True).set_index("timestamp")
    plot_prediction(i, y_pred)
    

SARIMA

In [ ]:
# #For a more complex version we will try SARIMA
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 4) for x in list(itertools.product(p, d, q))]

def sarima_validation(train, pdq, seasonal_pdq):
    best_aic, best_order, best_seasonal_order = float("inf"), float("inf"), None
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                sarimax_model = SARIMAX(train, order=param, seasonal_order=param_seasonal)
                results = sarimax_model.fit(disp=0)
                aic = results.aic
                if aic < best_aic:
                    best_aic, best_order, best_seasonal_order = aic, param, param_seasonal
                print('SARIMA{}x{}4 - AIC:{}'.format(param, param_seasonal, aic))
            except:
                continue
    print('SARIMA{}x{}4 - AIC:{}'.format(best_order, best_seasonal_order, best_aic))
    return best_order, best_seasonal_order  

for i in range(10):   
    best_order, best_seasonal_order = sarima_validation(train_data[i], pdq, seasonal_pdq)
    model = SARIMAX(train_data[i], order=best_order, seasonal_order=best_seasonal_order)
    sarima_final_model = model.fit(disp=0)
    y_pred_test = sarima_final_model.get_forecast(steps=142)

    y_pred = y_pred_test.predicted_mean
    mean_absolute_error(test_data[i], y_pred)
    y_pred.reset_index(drop=True,inplace=True)
    y_pred=pd.DataFrame(y_pred)
    y_pred = y_pred.merge(test_data[i].reset_index()["timestamp"], left_index=True, right_index=True).set_index("timestamp")
    plot_prediction(i, y_pred)

EXPONENTIAL SMOOTHING

In [ ]:
#First define the function that finds hyperparameters
def triple_validation(train,test, abg, step=142):
    best_alpha, best_beta, best_gamma, lowest_mae = None, None, None, float("inf")
    for params in abg:
        valid_model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=10).\
            fit(smoothing_level=params[0], smoothing_slope=params[1], smoothing_seasonal=params[2])
        y_pred = valid_model.forecast(step)
        mae = mean_absolute_error(test, y_pred)
        if mae < lowest_mae:
            best_alpha, best_beta, best_gamma, lowest_mae = params[0], params[1], params[2], mae
        print([round(params[0], 2), round(params[1], 2), round(params[2], 2), round(mae, 2)])

    print("Best_alpha is:", round(best_alpha, 2), " Best_beta is:", round(best_beta, 2), " Best_gamma is:", round(best_gamma, 2),
          "and Lowest_mae is:", round(lowest_mae, 4))

    return best_alpha, best_beta, best_gamma, lowest_mae   
    
alphas = betas = gammas = np.arange(0.01, 1, 0.20)
abg = list(itertools.product(alphas, betas, gammas))
for i in range(10):
    best_alpha, best_beta, best_gamma, _ = triple_validation(train_data[i], test_data[i], abg, step=142)    
    test_model = ExponentialSmoothing(train_data[i], trend="add", seasonal="add", seasonal_periods=90).fit(smoothing_level=best_alpha, smoothing_slope=best_beta, smoothing_seasonal=best_gamma)

    y_pred = test_model.forecast(142)    

    y_pred.reset_index(drop=True,inplace=True)
    y_pred=pd.DataFrame(y_pred, columns=["usage"])
    y_pred = y_pred.merge(test_data[i].reset_index()["timestamp"], left_index=True, right_index=True).set_index("timestamp")
    plot_prediction(i, y_pred)

In the end by looking at the results we see that none of the functions actually give the desired results. But the order from the best to the worst is Exponential Smoothing, SARIMA, Simple Exponential Smoothing.